# 07. 고급 Text2SQL 최적화

이 노트북에서는 Text2SQL 성능을 극대화하기 위한 고급 최적화 기법을 다룹니다:

- **Section 1**: 사내 문서 임베딩 시스템 - 다양한 문서 형식 지원 및 청킹 전략
- **Section 2**: 렉시콘 및 도메인 사전 관리 - 용어 매핑 및 동의어 확장
- **Section 3**: 데이터베이스 스키마 확장 - 컬럼 레벨 설명 및 비즈니스 로직
- **Section 4**: Few-shot 예제 관리 시스템 - 동적 예제 선택 및 품질 관리
- **Section 5**: 통합 파이프라인 - 전체 워크플로우 및 성능 모니터링

이 기법들을 통해 Text2SQL의 정확도와 신뢰성을 크게 향상시킬 수 있습니다.

## Section 1: 사내 문서 임베딩 시스템

다양한 문서 형식을 지원하고 효과적인 청킹 전략을 통해 사내 문서를 임베딩하여 Text2SQL 컨텍스트를 강화합니다.

### 1.1 라이브러리 임포트 및 초기화

In [ ]:
import sys
sys.path.append('/workspace')

from src.utils.db_utils import DatabaseConnection
from src.utils.document_loader import DocumentLoader, DocumentChunk
from src.utils.embedding_utils import generate_embedding, search_similar_documents
from src.utils.lexicon_manager import LexiconManager, initialize_default_lexicon
from src.utils.schema_enhancer import SchemaEnhancer, initialize_default_schema_descriptions
from src.utils.example_manager import ExampleManager, initialize_default_examples
from src.utils.text2sql_utils import Text2SQLGenerator

import pandas as pd
import os
from pathlib import Path

print("✓ 라이브러리 임포트 완료")

In [ ]:
# 데이터베이스 연결 초기화
db = DatabaseConnection()
print("✓ 데이터베이스 연결 완료")

# 문서 로더 초기화
doc_loader = DocumentLoader()
print("✓ 문서 로더 초기화 완료")
print(f"  지원 형식: {doc_loader.supported_formats}")

### 1.2 다양한 문서 형식 지원

PDF, Word, Excel, Markdown 등 다양한 형식의 문서를 로드할 수 있습니다.

In [ ]:
# 샘플 문서 생성 (예제용)
sample_docs_dir = Path('/workspace/data/sample_documents')
sample_docs_dir.mkdir(parents=True, exist_ok=True)

# 샘플 텍스트 문서 생성
sample_text = """회사 데이터베이스 사용 가이드

1. 직원 테이블 (employees)
직원 테이블에는 모든 직원의 정보가 저장되어 있습니다.
- employee_id: 직원 고유 번호
- name: 직원 이름
- salary: 연봉 (원 단위)
- hire_date: 입사일
- department_id: 소속 부서 번호

2. 부서 테이블 (departments)
회사의 모든 부서 정보를 관리합니다.
- department_id: 부서 고유 번호
- department_name: 부서명 (예: 개발팀, 영업팀, 마케팅팀)
- manager_id: 부서장 직원 번호

3. 매출 테이블 (sales)
모든 판매 거래 내역을 기록합니다.
- sale_id: 판매 거래 번호
- total_amount: 판매 금액
- sale_date: 판매 일자
- region: 판매 지역 (서울, 부산, 대구 등)
- customer_id: 고객 번호

비즈니스 용어 매핑:
- '급여', '연봉', '월급' → salary
- '매출', '판매액', '판매금액' → total_amount
- '직원', '사원', '임직원' → employees 테이블
- '부서', '부서명' → departments 테이블
"""

sample_file = sample_docs_dir / 'database_guide.txt'
with open(sample_file, 'w', encoding='utf-8') as f:
    f.write(sample_text)

print(f"✓ 샘플 문서 생성: {sample_file}")

In [ ]:
# 단일 문서 로드 예제
metadata = {
    'title': '데이터베이스 사용 가이드',
    'department': 'IT',
    'category': 'documentation',
    'priority': 'high'
}

chunks = doc_loader.load_document(str(sample_file), metadata=metadata)

print(f"\n✓ 문서 로드 완료: {len(chunks)}개의 청크 생성\n")
print("첫 번째 청크 정보:")
print(f"  내용 길이: {len(chunks[0].content)} 문자")
print(f"  메타데이터: {chunks[0].metadata}")
print(f"\n  내용 미리보기:\n{chunks[0].content[:200]}...")

### 1.3 문서 청킹 전략 비교

세 가지 청킹 전략을 비교합니다:
- **고정 크기 (fixed)**: 일정한 문자 수로 분할
- **문단 단위 (paragraph)**: 문단 경계를 기준으로 분할
- **의미론적 (semantic)**: 문장 단위로 의미를 유지하며 분할

In [ ]:
# 세 가지 청킹 전략 비교
chunk_size = 300

fixed_chunks = doc_loader.chunk_text(
    sample_text, 
    metadata={'strategy': 'fixed'}, 
    chunk_size=chunk_size, 
    strategy='fixed'
)

paragraph_chunks = doc_loader.chunk_text(
    sample_text, 
    metadata={'strategy': 'paragraph'}, 
    chunk_size=chunk_size, 
    strategy='paragraph'
)

semantic_chunks = doc_loader.chunk_text(
    sample_text, 
    metadata={'strategy': 'semantic'}, 
    chunk_size=chunk_size, 
    strategy='semantic'
)

print("청킹 전략별 결과 비교:")
print("=" * 60)
print(f"고정 크기 (Fixed): {len(fixed_chunks)}개 청크")
print(f"문단 단위 (Paragraph): {len(paragraph_chunks)}개 청크")
print(f"의미론적 (Semantic): {len(semantic_chunks)}개 청크")

print("\n각 전략의 첫 청크 길이:")
print(f"  Fixed: {len(fixed_chunks[0].content)} 문자")
print(f"  Paragraph: {len(paragraph_chunks[0].content)} 문자")
print(f"  Semantic: {len(semantic_chunks[0].content)} 문자")

In [ ]:
# 각 전략의 첫 번째 청크 비교
print("\n=== 고정 크기 청킹 (첫 청크) ===")
print(fixed_chunks[0].content[:200])

print("\n=== 문단 단위 청킹 (첫 청크) ===")
print(paragraph_chunks[0].content[:200])

print("\n=== 의미론적 청킹 (첫 청크) ===")
print(semantic_chunks[0].content[:200])

### 1.4 문서 임베딩 및 저장

청크된 문서를 임베딩하여 데이터베이스에 저장합니다.

In [ ]:
# documents 테이블에 청크 저장
def store_document_chunks(chunks, use_paragraph_strategy=True):
    """
    문서 청크를 임베딩하여 데이터베이스에 저장
    """
    # 문단 전략 사용 (가장 실용적)
    selected_chunks = paragraph_chunks if use_paragraph_strategy else chunks
    
    stored_count = 0
    
    for chunk in selected_chunks:
        try:
            # 임베딩 생성
            embedding = generate_embedding(chunk.content)
            
            # 데이터베이스에 저장
            query = """
            INSERT INTO documents (content, embedding, metadata)
            VALUES (%s, %s::vector, %s)
            """
            
            conn = db.get_connection()
            try:
                with conn.cursor() as cursor:
                    cursor.execute(
                        query,
                        (chunk.content, embedding, chunk.metadata)
                    )
                    conn.commit()
                    stored_count += 1
            finally:
                conn.close()
                
        except Exception as e:
            print(f"  ⚠ 청크 저장 실패: {str(e)[:100]}")
            continue
    
    return stored_count

# 문서 청크 저장
print("문서 청크 임베딩 및 저장 중...")
stored = store_document_chunks(paragraph_chunks)
print(f"✓ {stored}개 청크 저장 완료")

In [ ]:
# 저장된 문서 확인
query = "SELECT id, LEFT(content, 100) as content_preview, metadata FROM documents ORDER BY id DESC LIMIT 3"
result = db.execute_query_df(query)
print("\n최근 저장된 문서 청크:")
print(result)

### 1.5 문서 기반 컨텍스트 검색

사용자 질의와 유사한 문서를 검색하여 Text2SQL 컨텍스트를 강화합니다.

In [ ]:
# 유사 문서 검색 예제
user_query = "직원들의 급여 정보를 조회하고 싶어요"

print(f"사용자 질의: {user_query}\n")

# 유사 문서 검색
try:
    similar_docs = search_similar_documents(
        query_text=user_query,
        top_k=3,
        similarity_threshold=0.3
    )
    
    if similar_docs:
        print("관련 문서 검색 결과:")
        print("=" * 60)
        for i, doc in enumerate(similar_docs, 1):
            print(f"\n[{i}] 유사도: {doc.get('similarity', 0):.4f}")
            print(f"내용: {doc['content'][:150]}...")
    else:
        print("관련 문서를 찾지 못했습니다.")
        
except Exception as e:
    print(f"문서 검색 중 오류: {str(e)}")

## Section 2: 렉시콘 및 도메인 사전 관리

비즈니스 용어를 기술 용어로 매핑하여 Text2SQL의 정확도를 높입니다.

### 2.1 렉시콘 관리자 초기화

In [ ]:
# 렉시콘 관리자 초기화
lexicon = LexiconManager(db)
print("✓ 렉시콘 관리자 초기화 완료")

# 기본 용어 매핑 초기화
try:
    count = initialize_default_lexicon(db)
    print(f"✓ 기본 용어 매핑 {count}개 추가")
except Exception as e:
    print(f"  (용어 매핑이 이미 존재할 수 있습니다: {str(e)[:50]})")

### 2.2 용어 매핑 시스템

비즈니스 용어와 기술 용어를 매핑합니다.

In [ ]:
# 모든 용어 매핑 조회
mappings_df = lexicon.get_all_mappings()
print(f"전체 용어 매핑: {len(mappings_df)}개\n")
print(mappings_df[['business_term', 'technical_term', 'synonyms', 'category']].head(10))

In [ ]:
# 카테고리별 용어 조회
categories = lexicon.get_categories()
print(f"\n등록된 카테고리: {categories}\n")

for category in categories:
    cat_mappings = lexicon.get_all_mappings(category=category)
    print(f"{category}: {len(cat_mappings)}개 용어")

### 2.3 동의어 확장

다양한 동의어를 추가하여 자연어 질의를 더 유연하게 처리합니다.

In [ ]:
# 새로운 용어 매핑 추가
additional_terms = [
    {
        'business_term': '인원수',
        'technical_term': 'COUNT(*)',
        'synonyms': ['사람 수', '직원 수', '인원', '명수'],
        'description': '개수를 세는 집계 함수',
        'category': 'aggregation'
    },
    {
        'business_term': '평균',
        'technical_term': 'AVG',
        'synonyms': ['평균값', '평균금액', 'average'],
        'description': '평균을 계산하는 집계 함수',
        'category': 'aggregation'
    },
    {
        'business_term': '합계',
        'technical_term': 'SUM',
        'synonyms': ['총합', '전체', '총액', 'total'],
        'description': '합계를 계산하는 집계 함수',
        'category': 'aggregation'
    },
    {
        'business_term': '최고',
        'technical_term': 'MAX',
        'synonyms': ['최대', '가장 높은', '제일 큰'],
        'description': '최댓값을 찾는 함수',
        'category': 'aggregation'
    },
    {
        'business_term': '최저',
        'technical_term': 'MIN',
        'synonyms': ['최소', '가장 낮은', '제일 작은'],
        'description': '최솟값을 찾는 함수',
        'category': 'aggregation'
    }
]

try:
    added = lexicon.add_bulk_mappings(additional_terms)
    print(f"✓ {added}개의 추가 용어 매핑 등록")
except Exception as e:
    print(f"  (일부 용어가 이미 존재할 수 있습니다)")

### 2.4 자연어 질의 정규화

렉시콘을 사용하여 자연어 질의를 SQL 친화적으로 변환합니다.

In [ ]:
# 질의 정규화 예제
test_queries = [
    "매출이 가장 높은 직원 5명을 보여주세요",
    "부서별 평균 급여를 계산해주세요",
    "인원수가 가장 많은 부서는 어디인가요?",
    "최저 연봉을 받는 사원의 이름을 알려주세요",
    "지역별 총 판매액을 보여주세요"
]

print("자연어 질의 정규화 예제:")
print("=" * 80)

for query in test_queries:
    normalized, mappings = lexicon.normalize_query(query)
    print(f"\n원본: {query}")
    print(f"정규화: {normalized}")
    if mappings:
        print(f"적용된 매핑: {[m['original'] + ' → ' + m['replacement'] for m in mappings]}")

### 2.5 용어 검색 및 관리

In [ ]:
# 특정 용어 검색
search_term = "급여"
results = lexicon.search_terms(search_term, limit=5)

print(f"'{search_term}' 검색 결과:\n")
for result in results:
    print(f"비즈니스 용어: {result['business_term']}")
    print(f"기술 용어: {result['technical_term']}")
    print(f"동의어: {result['synonyms']}")
    print(f"카테고리: {result['category']}")
    print("-" * 40)

In [ ]:
# 용어를 기술 용어로 변환
business_terms = ["매출", "직원", "급여", "부서"]

print("비즈니스 용어 → 기술 용어 변환:\n")
for term in business_terms:
    technical = lexicon.get_technical_term(term)
    print(f"{term:10s} → {technical if technical else '(매핑 없음)'}")

## Section 3: 데이터베이스 스키마 확장

컬럼 레벨 설명과 비즈니스 로직을 추가하여 LLM이 스키마를 더 잘 이해하도록 합니다.

### 3.1 스키마 인핸서 초기화

In [ ]:
# 스키마 인핸서 초기화
schema_enhancer = SchemaEnhancer(db)
print("✓ 스키마 인핸서 초기화 완료")

# 기본 스키마 설명 초기화
try:
    count = initialize_default_schema_descriptions(db)
    print(f"✓ 기본 스키마 설명 {count}개 추가")
except Exception as e:
    print(f"  (스키마 설명이 이미 존재할 수 있습니다)")

### 3.2 컬럼 레벨 설명 추가

각 컬럼에 대한 상세한 설명과 비즈니스 의미를 추가합니다.

In [ ]:
# 특정 테이블의 컬럼 설명 조회
table_name = 'employees'
descriptions = schema_enhancer.get_table_descriptions(table_name)

print(f"{table_name} 테이블 컬럼 설명:\n")
print(descriptions[['column_name', 'description', 'business_meaning', 'example_values']])

In [ ]:
# 추가 컬럼 설명 등록
additional_descriptions = [
    {
        'table_name': 'projects',
        'column_name': 'project_id',
        'description': '프로젝트 고유 식별자',
        'business_meaning': '각 프로젝트를 구분하는 유일한 번호',
        'example_values': ['1', '2', '3'],
        'data_type': 'serial',
        'constraints': 'PRIMARY KEY'
    },
    {
        'table_name': 'projects',
        'column_name': 'project_name',
        'description': '프로젝트 이름',
        'business_meaning': '프로젝트의 공식 명칭',
        'example_values': ['신규 웹사이트 구축', 'AI 챗봇 개발', '데이터 분석 플랫폼'],
        'data_type': 'varchar(200)',
        'constraints': 'NOT NULL'
    },
    {
        'table_name': 'projects',
        'column_name': 'budget',
        'description': '프로젝트 예산',
        'business_meaning': '프로젝트에 할당된 총 예산 (원 단위)',
        'example_values': ['50000000', '100000000', '200000000'],
        'data_type': 'decimal',
        'constraints': 'CHECK (budget > 0)'
    },
    {
        'table_name': 'customers',
        'column_name': 'customer_name',
        'description': '고객 이름',
        'business_meaning': '개인 고객의 이름 또는 기업 고객의 회사명',
        'example_values': ['김철수', '이영희', '(주)테크컴퍼니'],
        'data_type': 'varchar(100)',
        'constraints': 'NOT NULL'
    }
]

try:
    added = schema_enhancer.add_bulk_descriptions(additional_descriptions)
    print(f"✓ {added}개의 추가 컬럼 설명 등록")
except Exception as e:
    print(f"  (일부 설명이 이미 존재할 수 있습니다)")

### 3.3 테이블 관계 설명

테이블 간 관계를 명확히 설명합니다.

In [ ]:
# 데이터베이스의 모든 테이블 조회
tables = db.get_all_tables()
print(f"데이터베이스 테이블 목록 ({len(tables)}개):\n")

for i, table in enumerate(tables, 1):
    # 기본 스키마 정보
    schema = db.get_table_schema(table)
    row_count_result = db.execute_query(f"SELECT COUNT(*) as count FROM {table}")
    row_count = row_count_result[0]['count'] if row_count_result else 0
    
    print(f"{i}. {table:20s} - {len(schema)} 컬럼, {row_count:5d} 행")

### 3.4 향상된 스키마 컨텍스트 생성

모든 설명을 통합하여 LLM을 위한 풍부한 스키마 컨텍스트를 생성합니다.

In [ ]:
# 특정 테이블의 향상된 스키마 조회
enhanced_schema = schema_enhancer.get_enhanced_schema('employees')

print("향상된 스키마 정보 (employees 테이블):\n")
print(enhanced_schema)

In [ ]:
# 전체 데이터베이스의 향상된 스키마 (처음 3개 테이블만)
full_schema = schema_enhancer.get_enhanced_schema()

print("전체 데이터베이스 향상된 스키마:\n")
# 출력이 너무 길어질 수 있으므로 처음 2000자만 표시
print(full_schema[:2000] + "\n...\n(생략)")

### 3.5 스키마 정보 내보내기/가져오기

In [ ]:
# 스키마 설명을 CSV로 내보내기
export_path = '/workspace/data/schema_descriptions.csv'
schema_enhancer.export_to_csv(export_path)
print(f"✓ 스키마 설명을 CSV로 내보냄: {export_path}")

# 내보낸 파일 확인
df = pd.read_csv(export_path)
print(f"\n총 {len(df)}개 컬럼 설명 내보냄")
print(df.head())

## Section 4: Few-shot 예제 관리 시스템

유사한 질의 예제를 검색하여 LLM에게 제공함으로써 SQL 생성 품질을 향상시킵니다.

### 4.1 예제 관리자 초기화

In [ ]:
# 예제 관리자 초기화
example_manager = ExampleManager(db)
print("✓ 예제 관리자 초기화 완료")

# 기본 예제 초기화
try:
    count = initialize_default_examples(db)
    print(f"✓ 기본 예제 {count}개 추가")
except Exception as e:
    print(f"  (예제가 이미 존재할 수 있습니다)")

### 4.2 예제 데이터베이스 구축

다양한 유형의 질의 예제를 추가합니다.

In [ ]:
# 저장된 예제 조회
all_examples = example_manager.search_examples(limit=100)
print(f"전체 예제 수: {len(all_examples)}개\n")

# 카테고리별 분포
categories = example_manager.get_categories()
print(f"예제 카테고리: {categories}\n")

for category in categories:
    cat_examples = example_manager.search_examples(category=category, limit=100)
    print(f"  {category:15s}: {len(cat_examples):2d}개")

In [ ]:
# 난이도별 예제 조회
difficulties = ['easy', 'medium', 'hard']

print("\n난이도별 예제 분포:\n")
for difficulty in difficulties:
    examples = example_manager.search_examples(difficulty=difficulty, limit=100)
    print(f"  {difficulty:10s}: {len(examples):2d}개")

In [ ]:
# 추가 고급 예제 등록
advanced_examples = [
    {
        'natural_language_query': '각 부서에서 급여가 가장 높은 직원의 이름과 급여를 보여주세요',
        'sql_query': '''SELECT d.department_name, e.name, e.salary
FROM employees e
JOIN departments d ON e.department_id = d.department_id
WHERE (e.department_id, e.salary) IN (
    SELECT department_id, MAX(salary)
    FROM employees
    GROUP BY department_id
);''',
        'query_category': 'subquery',
        'difficulty': 'hard',
        'tags': ['subquery', 'max', 'groupby', 'join']
    },
    {
        'natural_language_query': '2024년 1월의 지역별 매출과 전월 대비 증감률을 보여주세요',
        'sql_query': '''WITH current_month AS (
    SELECT region, SUM(total_amount) as current_sales
    FROM sales
    WHERE sale_date >= '2024-01-01' AND sale_date < '2024-02-01'
    GROUP BY region
),
previous_month AS (
    SELECT region, SUM(total_amount) as previous_sales
    FROM sales
    WHERE sale_date >= '2023-12-01' AND sale_date < '2024-01-01'
    GROUP BY region
)
SELECT c.region, c.current_sales, p.previous_sales,
       ROUND((c.current_sales - p.previous_sales) * 100.0 / p.previous_sales, 2) as growth_rate
FROM current_month c
LEFT JOIN previous_month p ON c.region = p.region;''',
        'query_category': 'window',
        'difficulty': 'hard',
        'tags': ['cte', 'comparison', 'calculation', 'date']
    },
    {
        'natural_language_query': '프로젝트에 참여하지 않는 직원들을 보여주세요',
        'sql_query': '''SELECT e.* 
FROM employees e
WHERE e.employee_id NOT IN (
    SELECT DISTINCT employee_id 
    FROM project_assignments
);''',
        'query_category': 'subquery',
        'difficulty': 'medium',
        'tags': ['not in', 'subquery', 'exclusion']
    },
    {
        'natural_language_query': '부서별로 급여 상위 3명의 직원을 보여주세요',
        'sql_query': '''WITH ranked_employees AS (
    SELECT e.*, d.department_name,
           ROW_NUMBER() OVER (PARTITION BY e.department_id ORDER BY e.salary DESC) as rank
    FROM employees e
    JOIN departments d ON e.department_id = d.department_id
)
SELECT department_name, name, salary, rank
FROM ranked_employees
WHERE rank <= 3
ORDER BY department_name, rank;''',
        'query_category': 'window',
        'difficulty': 'hard',
        'tags': ['window function', 'rank', 'partition', 'cte']
    },
    {
        'natural_language_query': '고객별 총 구매금액과 구매횟수를 보여주세요',
        'sql_query': '''SELECT c.customer_name, 
       COUNT(s.sale_id) as purchase_count,
       SUM(s.total_amount) as total_amount,
       AVG(s.total_amount) as avg_amount
FROM customers c
LEFT JOIN sales s ON c.customer_id = s.customer_id
GROUP BY c.customer_id, c.customer_name
ORDER BY total_amount DESC;''',
        'query_category': 'aggregation',
        'difficulty': 'medium',
        'tags': ['aggregation', 'join', 'groupby', 'multiple metrics']
    }
]

try:
    added = example_manager.add_bulk_examples(advanced_examples)
    print(f"✓ {added}개의 고급 예제 추가")
except Exception as e:
    print(f"  (일부 예제가 이미 존재할 수 있습니다)")

### 4.3 유사 질의 검색 (임베딩 기반)

사용자 질의와 유사한 예제를 검색하여 few-shot 프롬프트를 구성합니다.

In [ ]:
# 예제 임베딩 생성
print("예제 임베딩 생성 중...")
try:
    example_manager.generate_embeddings_for_all()
    print("✓ 임베딩 생성 완료")
except Exception as e:
    print(f"⚠ 임베딩 생성 중 오류: {str(e)[:100]}")
    print("  (임베딩 없이 키워드 검색을 사용합니다)")

In [ ]:
# 유사 예제 검색 테스트
test_queries = [
    "급여가 높은 직원들을 보여주세요",
    "부서별 직원 수를 계산해주세요",
    "가장 많이 판매된 지역은 어디인가요?",
    "프로젝트에 배정된 직원 목록을 보여주세요"
]

print("유사 예제 검색 테스트:")
print("=" * 80)

for test_query in test_queries:
    print(f"\n사용자 질의: {test_query}")
    print("-" * 80)
    
    try:
        similar_examples = example_manager.find_similar_examples(
            natural_query=test_query,
            limit=3,
            similarity_threshold=0.3
        )
        
        if similar_examples:
            for i, (example, similarity) in enumerate(similar_examples, 1):
                print(f"\n  [{i}] 유사도: {similarity:.4f}")
                print(f"      질의: {example['natural_language_query']}")
                print(f"      SQL: {example['sql_query'][:80]}...")
        else:
            print("  유사한 예제를 찾지 못했습니다.")
            
    except Exception as e:
        print(f"  오류: {str(e)[:100]}")

### 4.4 예제 품질 관리

예제의 성공률과 사용 빈도를 추적하여 품질을 관리합니다.

In [ ]:
# 성공률이 높은 예제 조회
top_examples = example_manager.get_top_examples(limit=5)

print("성공률이 높은 예제 Top 5:\n")
for i, example in enumerate(top_examples, 1):
    print(f"{i}. {example['natural_language_query']}")
    print(f"   성공률: {example['success_rate']:.2%}, 사용 횟수: {example['usage_count']}회")
    print(f"   카테고리: {example['query_category']}, 난이도: {example['difficulty']}")
    print()

In [ ]:
# 카테고리별 Top 예제
for category in ['aggregation', 'join', 'filter']:
    cat_top = example_manager.get_top_examples(category=category, limit=2)
    
    if cat_top:
        print(f"\n[{category}] 카테고리 Top 예제:")
        for example in cat_top:
            print(f"  - {example['natural_language_query']}")
            print(f"    성공률: {example['success_rate']:.2%}")

### 4.5 동적 Few-shot 선택

질의 유형에 따라 가장 적절한 예제를 동적으로 선택합니다.

In [ ]:
def select_best_examples(user_query: str, n_examples: int = 3) -> list:
    """
    사용자 질의에 가장 적합한 예제를 선택
    
    전략:
    1. 임베딩 기반 유사도 검색
    2. 성공률이 높은 예제 우선
    3. 다양한 카테고리 포함
    """
    # 유사한 예제 검색
    similar = example_manager.find_similar_examples(
        natural_query=user_query,
        limit=n_examples * 2,  # 더 많이 검색하여 필터링
        similarity_threshold=0.2
    )
    
    if not similar:
        # 유사 예제가 없으면 성공률이 높은 예제 사용
        top_examples = example_manager.get_top_examples(limit=n_examples)
        return [(ex, 0.0) for ex in top_examples]
    
    # 성공률과 유사도를 결합한 점수로 정렬
    scored_examples = []
    for example, similarity in similar:
        # 점수 = (유사도 * 0.7) + (성공률 * 0.3)
        score = (similarity * 0.7) + (example.get('success_rate', 0.5) * 0.3)
        scored_examples.append((example, similarity, score))
    
    # 점수 순으로 정렬
    scored_examples.sort(key=lambda x: x[2], reverse=True)
    
    # 상위 n개 선택 (다양성 고려)
    selected = []
    seen_categories = set()
    
    for example, similarity, score in scored_examples:
        if len(selected) >= n_examples:
            break
        
        category = example.get('query_category')
        
        # 카테고리 다양성 확보 (이미 2개 이상이면 스킵)
        if category and seen_categories.count(category) >= 2:
            continue
        
        selected.append((example, similarity))
        if category:
            seen_categories.add(category)
    
    # 부족하면 나머지 채우기
    if len(selected) < n_examples:
        for example, similarity, score in scored_examples:
            if len(selected) >= n_examples:
                break
            if (example, similarity) not in selected:
                selected.append((example, similarity))
    
    return selected

# 테스트
test_query = "부서별 평균 급여와 최고 급여를 보여주세요"
print(f"사용자 질의: {test_query}\n")

best_examples = select_best_examples(test_query, n_examples=3)

print("선택된 Few-shot 예제:\n")
for i, (example, similarity) in enumerate(best_examples, 1):
    print(f"[예제 {i}] (유사도: {similarity:.4f})")
    print(f"질의: {example['natural_language_query']}")
    print(f"SQL: {example['sql_query'][:100]}...")
    print()

## Section 5: 통합 파이프라인

모든 최적화 기법을 통합하여 완전한 Text2SQL 파이프라인을 구축합니다.

### 5.1 최적화된 Text2SQL 생성기

In [ ]:
class OptimizedText2SQL:
    """
    모든 최적화 기법을 통합한 Text2SQL 생성기
    """
    
    def __init__(self, db, llm_provider="ollama", model_name="llama2"):
        self.db = db
        self.lexicon = LexiconManager(db)
        self.schema_enhancer = SchemaEnhancer(db)
        self.example_manager = ExampleManager(db)
        
        try:
            self.generator = Text2SQLGenerator(llm_provider=llm_provider, model_name=model_name)
        except:
            self.generator = None
    
    def generate_sql(self, natural_query: str, use_examples: bool = True, n_examples: int = 3) -> dict:
        """
        자연어 질의를 SQL로 변환 (모든 최적화 적용)
        
        Args:
            natural_query: 사용자의 자연어 질의
            use_examples: Few-shot 예제 사용 여부
            n_examples: 사용할 예제 수
        
        Returns:
            {
                'original_query': 원본 질의,
                'normalized_query': 정규화된 질의,
                'sql': 생성된 SQL,
                'examples_used': 사용된 예제 목록,
                'context_documents': 관련 문서,
                'success': 성공 여부
            }
        """
        result = {
            'original_query': natural_query,
            'normalized_query': None,
            'sql': None,
            'examples_used': [],
            'context_documents': [],
            'applied_mappings': [],
            'success': False,
            'error': None
        }
        
        try:
            # 1. 용어 정규화
            normalized_query, mappings = self.lexicon.normalize_query(natural_query)
            result['normalized_query'] = normalized_query
            result['applied_mappings'] = mappings
            
            # 2. 관련 문서 검색
            try:
                docs = search_similar_documents(natural_query, top_k=2, similarity_threshold=0.3)
                result['context_documents'] = docs
            except:
                pass
            
            # 3. 유사 예제 선택
            if use_examples:
                similar_examples = self.example_manager.find_similar_examples(
                    natural_query=normalized_query,
                    limit=n_examples,
                    similarity_threshold=0.3
                )
                result['examples_used'] = [(ex, sim) for ex, sim in similar_examples]
            
            # 4. 향상된 스키마 컨텍스트
            enhanced_schema = self.schema_enhancer.get_enhanced_schema()
            
            # 5. SQL 생성 (실제 LLM 사용)
            if self.generator:
                # Few-shot 예제 준비
                examples_text = ""
                if result['examples_used']:
                    examples_text = "\n\nRelevant Examples:\n"
                    for i, (ex, sim) in enumerate(result['examples_used'], 1):
                        examples_text += f"\nExample {i}:\n"
                        examples_text += f"Q: {ex['natural_language_query']}\n"
                        examples_text += f"SQL: {ex['sql_query']}\n"
                
                # 프롬프트 구성
                prompt = f"""
Given the following database schema and examples, generate a SQL query for the user's question.

{enhanced_schema[:1500]}

{examples_text}

User Question: {normalized_query}

Generate only the SQL query, without explanations.
"""
                
                sql = self.generator.generate_sql(prompt)
                result['sql'] = sql
                result['success'] = True
            else:
                # LLM 없이 간단한 매핑 기반 생성 (예제용)
                result['sql'] = "-- LLM not available. Please configure Ollama."
                result['success'] = False
                
        except Exception as e:
            result['error'] = str(e)
            result['success'] = False
        
        return result

# 최적화된 Text2SQL 생성기 초기화
optimized_generator = OptimizedText2SQL(db)
print("✓ 최적화된 Text2SQL 생성기 초기화 완료")

### 5.2 전체 워크플로우 실행

In [ ]:
# 테스트 질의들
test_queries = [
    "급여가 가장 높은 직원 3명을 보여주세요",
    "부서별 평균 연봉을 계산해주세요",
    "서울 지역의 총 매출은 얼마인가요?",
    "프로젝트에 참여하는 직원들의 명단을 보여주세요"
]

print("최적화된 Text2SQL 파이프라인 실행:\n")
print("=" * 80)

for query in test_queries:
    print(f"\n질의: {query}")
    print("-" * 80)
    
    result = optimized_generator.generate_sql(query, use_examples=True, n_examples=2)
    
    # 정규화 결과
    if result['applied_mappings']:
        print(f"\n정규화: {result['normalized_query']}")
        print(f"적용된 매핑: {[m['original'] + '→' + m['replacement'] for m in result['applied_mappings']]}")
    
    # 사용된 예제
    if result['examples_used']:
        print(f"\n유사 예제 {len(result['examples_used'])}개 사용:")
        for i, (ex, sim) in enumerate(result['examples_used'], 1):
            print(f"  {i}. {ex['natural_language_query']} (유사도: {sim:.3f})")
    
    # 관련 문서
    if result['context_documents']:
        print(f"\n관련 문서 {len(result['context_documents'])}개 발견")
    
    # 생성된 SQL
    if result['success'] and result['sql']:
        print(f"\n생성된 SQL:\n{result['sql']}")
    else:
        print(f"\nSQL 생성 실패: {result.get('error', 'Unknown error')}")
    
    print()

### 5.3 성능 모니터링

Text2SQL 시스템의 성능을 추적하고 모니터링합니다.

In [ ]:
# 쿼리 히스토리 조회
def get_query_statistics():
    """
    Text2SQL 쿼리 통계 조회
    """
    stats = {}
    
    try:
        # 전체 쿼리 수
        total_query = "SELECT COUNT(*) as count FROM query_history"
        result = db.execute_query(total_query)
        stats['total_queries'] = result[0]['count'] if result else 0
        
        # 성공/실패 비율
        success_query = "SELECT success, COUNT(*) as count FROM query_history GROUP BY success"
        result = db.execute_query(success_query)
        stats['success_rate'] = {}
        for row in result:
            stats['success_rate'][row['success']] = row['count']
        
        # 최근 쿼리
        recent_query = "SELECT * FROM query_history ORDER BY timestamp DESC LIMIT 5"
        stats['recent_queries'] = db.execute_query(recent_query)
        
    except Exception as e:
        stats['error'] = str(e)
    
    return stats

# 통계 조회
stats = get_query_statistics()

print("Text2SQL 성능 통계:\n")
print("=" * 60)

if 'error' not in stats:
    print(f"총 쿼리 수: {stats['total_queries']}")
    
    if stats['success_rate']:
        print("\n성공/실패 비율:")
        for success, count in stats['success_rate'].items():
            status = '성공' if success else '실패'
            print(f"  {status}: {count}개")
    
    if stats['recent_queries']:
        print("\n최근 쿼리:")
        for i, query in enumerate(stats['recent_queries'], 1):
            status = '✓' if query.get('success') else '✗'
            print(f"  {i}. {status} {query.get('natural_query', '')[:50]}...")
else:
    print(f"통계 조회 오류: {stats['error']}")
    print("(query_history 테이블이 아직 생성되지 않았을 수 있습니다)")

### 5.4 지속적 개선

시스템을 지속적으로 개선하기 위한 메커니즘을 구축합니다.

In [ ]:
def analyze_failed_queries():
    """
    실패한 쿼리를 분석하여 개선 포인트 도출
    """
    try:
        query = """
        SELECT natural_query, generated_sql, error_message
        FROM query_history
        WHERE success = FALSE
        ORDER BY timestamp DESC
        LIMIT 10
        """
        
        failed = db.execute_query(query)
        
        if not failed:
            print("실패한 쿼리가 없습니다.")
            return
        
        print(f"최근 실패한 쿼리 분석 ({len(failed)}개):\n")
        
        # 오류 유형 분석
        error_types = {}
        for item in failed:
            error = item.get('error_message', 'Unknown')
            error_type = error.split(':')[0] if ':' in error else error
            error_types[error_type] = error_types.get(error_type, 0) + 1
        
        print("오류 유형 분포:")
        for error_type, count in sorted(error_types.items(), key=lambda x: x[1], reverse=True):
            print(f"  - {error_type}: {count}회")
        
        print("\n개선 제안:")
        print("  1. 실패한 쿼리를 수동으로 수정하여 예제 데이터베이스에 추가")
        print("  2. 새로운 용어 매핑 추가")
        print("  3. 스키마 설명 보완")
        print("  4. 프롬프트 엔지니어링 개선")
        
    except Exception as e:
        print(f"분석 중 오류: {str(e)}")

# 실패 쿼리 분석
analyze_failed_queries()

In [ ]:
def suggest_new_examples():
    """
    자주 사용되는 쿼리 패턴을 분석하여 새로운 예제 추가 제안
    """
    print("쿼리 패턴 분석 및 예제 추가 제안:\n")
    
    # 카테고리별 예제 수 확인
    categories = example_manager.get_categories()
    
    category_counts = {}
    for category in categories:
        examples = example_manager.search_examples(category=category, limit=100)
        category_counts[category] = len(examples)
    
    if category_counts:
        print("카테고리별 예제 분포:")
        for category, count in sorted(category_counts.items(), key=lambda x: x[1]):
            print(f"  {category:15s}: {count:2d}개")
        
        # 예제가 부족한 카테고리
        min_count = min(category_counts.values())
        lacking = [cat for cat, cnt in category_counts.items() if cnt <= min_count + 2]
        
        if lacking:
            print(f"\n예제 추가가 필요한 카테고리: {', '.join(lacking)}")
    else:
        print("카테고리 정보 없음")
    
    print("\n권장 사항:")
    print("  1. 복잡한 JOIN 쿼리 예제 추가")
    print("  2. Window Function 사용 예제 추가")
    print("  3. CTE(Common Table Expression) 예제 추가")
    print("  4. 날짜/시간 관련 쿼리 예제 추가")

# 예제 추가 제안
suggest_new_examples()

### 5.5 최적화 효과 비교

In [ ]:
# 최적화 전후 비교
comparison_query = "부서별 평균 급여를 계산해주세요"

print("최적화 기법 적용 효과 비교:\n")
print("=" * 80)
print(f"질의: {comparison_query}\n")

# 1. 기본 처리 (최적화 없음)
print("[1] 기본 처리 (최적화 없음)")
print("-" * 80)
print(f"입력: {comparison_query}")
print("컨텍스트: 기본 스키마만 사용")
print("예제: 없음")
print()

# 2. 용어 정규화 적용
print("[2] 용어 정규화 적용")
print("-" * 80)
normalized, mappings = lexicon.normalize_query(comparison_query)
print(f"입력: {comparison_query}")
print(f"정규화: {normalized}")
if mappings:
    print(f"매핑: {[m['original'] + ' → ' + m['replacement'] for m in mappings]}")
print()

# 3. Few-shot 예제 추가
print("[3] Few-shot 예제 추가")
print("-" * 80)
similar = example_manager.find_similar_examples(normalized, limit=2, similarity_threshold=0.3)
print(f"유사 예제 {len(similar)}개 발견:")
for i, (ex, sim) in enumerate(similar, 1):
    print(f"  {i}. {ex['natural_language_query']} (유사도: {sim:.3f})")
print()

# 4. 스키마 컨텍스트 강화
print("[4] 스키마 컨텍스트 강화")
print("-" * 80)
enhanced = schema_enhancer.get_enhanced_schema('employees')
print("컬럼별 상세 설명, 비즈니스 의미, 예시 값 포함")
print(f"스키마 정보 크기: {len(enhanced)} 문자")
print()

# 5. 문서 컨텍스트 추가
print("[5] 문서 컨텍스트 추가")
print("-" * 80)
try:
    docs = search_similar_documents(comparison_query, top_k=2, similarity_threshold=0.3)
    print(f"관련 문서 {len(docs)}개 발견")
    for i, doc in enumerate(docs, 1):
        print(f"  {i}. {doc['content'][:60]}... (유사도: {doc.get('similarity', 0):.3f})")
except:
    print("관련 문서 없음")
print()

print("=" * 80)
print("\n최적화 효과:")
print("  ✓ 용어 정규화로 비즈니스 용어를 정확한 기술 용어로 변환")
print("  ✓ Few-shot 예제로 LLM에게 명확한 패턴 제공")
print("  ✓ 향상된 스키마로 테이블/컬럼의 비즈니스 의미 전달")
print("  ✓ 관련 문서로 도메인 지식 보강")
print("\n→ 이러한 최적화를 통해 SQL 생성 정확도와 품질이 크게 향상됩니다.")

## 요약 및 결론

이 노트북에서 다룬 고급 최적화 기법:

### 1. 사내 문서 임베딩 시스템
- ✓ PDF, Word, Excel, Markdown 등 다양한 형식 지원
- ✓ 고정 크기, 문단 단위, 의미론적 청킹 전략
- ✓ 메타데이터 관리 및 검색
- ✓ 벡터 유사도 기반 문서 검색

### 2. 렉시콘 및 도메인 사전 관리
- ✓ 비즈니스 용어 ↔ 기술 용어 매핑
- ✓ 동의어 확장 및 자동 정규화
- ✓ 카테고리별 용어 관리
- ✓ CSV 기반 대량 가져오기/내보내기

### 3. 데이터베이스 스키마 확장
- ✓ 컬럼별 상세 설명 및 비즈니스 의미
- ✓ 예시 값과 제약 조건 명시
- ✓ 테이블 관계 설명
- ✓ LLM을 위한 향상된 컨텍스트 생성

### 4. Few-shot 예제 관리 시스템
- ✓ 질의-SQL 쌍의 예제 데이터베이스
- ✓ 임베딩 기반 유사 예제 검색
- ✓ 성공률 및 사용 빈도 추적
- ✓ 동적 예제 선택 알고리즘

### 5. 통합 파이프라인
- ✓ 모든 최적화 기법 통합
- ✓ 성능 모니터링 및 통계
- ✓ 지속적 개선 메커니즘
- ✓ 실패 분석 및 피드백 루프

### 기대 효과

이러한 최적화를 통해:
1. **정확도 향상**: 용어 매핑과 풍부한 컨텍스트로 SQL 생성 정확도 증가
2. **일관성 확보**: Few-shot 예제로 일관된 쿼리 패턴 유지
3. **도메인 적응**: 사내 문서와 렉시콘으로 특정 도메인에 최적화
4. **유지보수성**: 체계적인 관리로 시스템 개선 및 확장 용이

### 다음 단계

- 실제 비즈니스 데이터로 시스템 학습
- 사용자 피드백 수집 및 반영
- A/B 테스트로 최적화 효과 측정
- 프로덕션 환경 배포 및 모니터링

## 추가 리소스

- 유틸리티 모듈:
  - `src/utils/document_loader.py` - 문서 로딩 및 청킹
  - `src/utils/lexicon_manager.py` - 용어 사전 관리
  - `src/utils/schema_enhancer.py` - 스키마 강화
  - `src/utils/example_manager.py` - 예제 관리
  
- 관련 노트북:
  - `02_embedding_and_rag.ipynb` - 임베딩 및 RAG 기초
  - `03_text2sql_basic.ipynb` - Text2SQL 기초
  - `04_agent_workflow.ipynb` - 에이전트 워크플로우
  - `06_end_to_end.ipynb` - 엔드투엔드 파이프라인